In [1]:
# script to compute ignition delays for 50 mechanisms of top parameters

In [7]:
import os
import cantera as ct
import numpy as np
import pandas as pd
import concurrent.futures


import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load the experimental conditions
ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
ignition_delay_data = '/home/moon/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
df_exp = pd.read_csv(ignition_delay_data)

# slice just table 7, where phi=1.0
table7 = df_exp[df_exp['Table'] == 7]
# Define Initial conditions using experimental data
tau7 = table7['time (ms)'].values.astype(float)  # ignition delay
T7 = table7['T_C'].values  # Temperatures
P7 = table7['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm


# list of starting conditions
# Mixture compositions taken from table 2 of
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
concentrations = []
# for phi = 1
x_diluent = 0.7649
conc_dict = {
    'O2(2)': 0.2038,
    'butane(1)': 0.03135
}

for i in range(0, len(table7)):
    x_N2 = table7['%N2'].values[i] / 100.0 * x_diluent
    x_Ar = table7['%Ar'].values[i] / 100.0 * x_diluent
    x_CO2 = table7['%CO2'].values[i] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    concentrations.append(conc_dict)


In [5]:
# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def get_delay(gas, T, P, X):
    # function to run a RCM simulation

    t_end = 1.0  # time in seconds
    gas.TPX = T, P, X

    env = ct.Reservoir(ct.Solution('air.yaml'))
    # env = ct.Reservoir(ct.Solution('air.xml'))
    reactor = ct.IdealGasReactor(gas)
    wall = ct.Wall(reactor, env, A=1.0, velocity=0)
    reactor_net = ct.ReactorNet([reactor])
    # # allegedly faster solving
    # reactor_net.derivative_settings = {"skip-third-bodies": True, "skip-falloff": True}
    # reactor_net.preconditioner = ct.AdaptivePreconditioner()

    times = [0]
    T = [reactor.T]
    P = [reactor.thermo.P]
    X = [reactor.thermo.X]  # mol fractions
    while reactor_net.time < t_end:
        reactor_net.step()

        times.append(reactor_net.time)
        T.append(reactor.T)
        P.append(reactor.thermo.P)
        X.append(reactor.thermo.X)

    slopes = np.gradient(P, times)
    i = np.argmax(slopes)
    return times[i]

In [6]:
def run_simulation(condition_index):
    gas = ct.Solution(cti_path)
    X = concentrations[condition_index]
    delay = get_delay(gas, T7[condition_index], P7[condition_index], X)
    print(f'Completed {condition_index}:\t {delay}')
    return delay

In [9]:
save_dir = '/home/moon/autoscience/autoscience/paper/error_vs_num_included/results'
mech_dir = '/home/moon/autoscience/autoscience/paper/error_vs_num_included/mechs'

In [10]:
for i in range(0, 51):
    print(f'running mechanism {i}')
    # load the CTI
    cti_path = os.path.join(mech_dir, f'top_{i}.cti')
    gas = ct.Solution(cti_path)
    
    # Run all simulations in parallel
    delays = np.zeros(len(table7))
    condition_indices = np.arange(0, len(table7))
    with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
        for condition_index, delay_time in zip(condition_indices, executor.map(run_simulation, condition_indices)):
            delays[condition_index] = delay_time
    
    np.save(os.path.join(save_dir, f'delays_{i}.npy'), delays)

running mechanism 0
Completed 11:	 0.05563236520895804Completed 8:	 0.10859170352272628

Completed 14:	 0.023349025151091538
Completed 13:	 0.024629607815479616
Completed 15:	 0.013983329164772643
Completed 6:	 0.14791667636256164
Completed 12:	 0.03535775087192712
Completed 4:	 0.07558799561746257Completed 10:	 0.06520822884244994

Completed 9:	 0.09524339850848947
Completed 7:	 0.1485753452189033
Completed 3:	 0.04868176194585489
Completed 5:	 0.07737167394281456
Completed 2:	 0.048144458947842486Completed 1:	 0.04854438699501166

Completed 0:	 0.049083278068407024
running mechanism 1
Completed 15:	 0.013996607920455895
Completed 12:	 0.03530453377130646
Completed 10:	 0.06452572816844132
Completed 13:	 0.024635717316313213
Completed 9:	 0.09277371288218411Completed 6:	 0.1246939937491167Completed 14:	 0.02335823585615054


Completed 8:	 0.10469297343500113
Completed 11:	 0.05524446979764865
Completed 7:	 0.1257927185200846
Completed 3:	 0.04901572340484172
Completed 5:	 0.0680784528

Completed 0:	 0.048273163162283944
Completed 8:	 0.09655472685789622
Completed 15:	 0.013803201191601288Completed 9:	 0.08707846545133373Completed 10:	 0.062261177020921626


Completed 6:	 0.098306254196392
Completed 5:	 0.055813672811295616
Completed 7:	 0.0994106855837579
Completed 3:	 0.04197815512983999
Completed 4:	 0.054952786574588866
Completed 1:	 0.04762722553535312
Completed 2:	 0.041762311762275496
running mechanism 15
Completed 9:	 0.0870747989871645
Completed 13:	 0.024256084194248045
Completed 15:	 0.013802588096891836
Completed 14:	 0.023005904773627527
Completed 12:	 0.03463334539680122
Completed 10:	 0.06225858848566495
Completed 7:	 0.0994028941513155
Completed 11:	 0.05364113965039343
Completed 4:	 0.054941990711988395
Completed 8:	 0.09655053597172696Completed 2:	 0.041753581395274506

Completed 1:	 0.04762209486464813
Completed 5:	 0.05580287309702339
Completed 6:	 0.09829837204142149
Completed 0:	 0.04826813025936605
Completed 3:	 0.04196929814944226
running mecha

Completed 6:	 0.09795286746608099
Completed 7:	 0.09902283666226938
Completed 3:	 0.0501876744010578
Completed 4:	 0.05846738041694752
Completed 5:	 0.059137523595834046
Completed 2:	 0.05015845947472253
Completed 0:	 0.06244372916419883
Completed 1:	 0.0615954369518701
running mechanism 29
Completed 11:	 0.05366585036336448
Completed 15:	 0.013748888944035613
Completed 14:	 0.0229865624471866
Completed 13:	 0.024241289409917703
Completed 6:	 0.09798308132269382
Completed 10:	 0.062254023073487715
Completed 12:	 0.03465048831468239Completed 9:	 0.08682296859905805

Completed 7:	 0.09905134695118689
Completed 8:	 0.09612358736829342
Completed 5:	 0.05931694667125518
Completed 3:	 0.050423359978368436
Completed 4:	 0.05865046975423196
Completed 2:	 0.0503945288195622
Completed 0:	 0.06264972103883633
Completed 1:	 0.06180329451477746
running mechanism 30
Completed 14:	 0.022988942446496078
Completed 12:	 0.034653645189292585
Completed 15:	 0.013750288715380395
Completed 11:	 0.0536699507

Completed 3:	 0.036390418853613486
Completed 1:	 0.05074970042020011
Completed 2:	 0.036543565962512646
Completed 4:	 0.03802088337196897
Completed 0:	 0.05161143653921929
running mechanism 43
Completed 11:	 0.04832536247053643
Completed 15:	 0.013349980117965045
Completed 10:	 0.05510943850230538
Completed 12:	 0.03232850670612388
Completed 14:	 0.021915103574382273
Completed 9:	 0.07245250627918526Completed 13:	 0.023056581172137203

Completed 8:	 0.07773079397739042
Completed 6:	 0.06233078793116345
Completed 7:	 0.06325819694043666
Completed 5:	 0.038312926811012345
Completed 3:	 0.036390418853613486
Completed 2:	 0.036543565962512646
Completed 4:	 0.03802088337196897
Completed 1:	 0.05074970042020011
Completed 0:	 0.05161143653921929
running mechanism 44
Completed 15:	 0.013349980117965045
Completed 13:	 0.023056581172137203
Completed 14:	 0.021915103574382273
Completed 12:	 0.03232850670612388
Completed 11:	 0.04832536247053643
Completed 10:	 0.05510943850230538
Completed 8:	 0.0